In [1]:
import gc
from sklearn.decomposition import NMF

gc.enable()

In [2]:
import pickle

with open("Data/df.pkl", 'rb') as picklefile:
    df = pickle.load(picklefile)

Let's do our analysis on stemmed words so that same topic is not split (e.g. "word" and "words" should belong to the same topic):

In [3]:
# import nltk
# from textblob import TextBlob
# stemmer = nltk.stem.porter.PorterStemmer()

# def stem_getter(text):
#     return " ".join([stemmer.stem(word) for word in TextBlob(text).words])

# df.raw_text = df.raw_text.map(stem_getter)

This time around, let's remove words in all caps: they are used to indicate character lines. Using them will just create topics identifying major characters of a show/movie which is not helpful. Let's also remove non-letter characters along the way:

In [4]:
import re

In [5]:
def cap_remover(text):
    text = re.sub(r'[A-Z]+(?![a-z])', '', text)
    text = re.sub(r'[\d]+', '', text)
    text = re.sub(r' +', ' ', text)
    return re.sub(r"[^\w' ]", '', text)

In [6]:
df.raw_text = df.raw_text.map(cap_remover)

In [7]:
%pylab inline
import numpy 
import matplotlib.pyplot as plt
import sklearn
# Import all of the scikit learn stuff
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn import metrics
import pandas as pd

Populating the interactive namespace from numpy and matplotlib


I have also used a large list of keywords from [here](http://www.ranks.nl/stopwords) and supplemented it with Star Trek specific terms discovered in the initial LDA model so that they are not used in topic analysis:

In [8]:
with open('Data/stopwords.txt') as f:
     content = (f.read()).split()
stoplist = sorted(list(set(content)))

In [9]:
from helper import *
import warnings
warnings.filterwarnings('ignore')

In [10]:
import nltk
# nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/aleksod/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
# from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string

stopwords = stoplist #set(stopwords.words('english'))
punctuation = set(string.punctuation) 
lemmatize = WordNetLemmatizer()

def cleaning(article):
    one = " ".join([i for i in article.lower().split() if i not in stopwords])
    two = "".join(i for i in one if i not in punctuation)
    three = " ".join(lemmatize.lemmatize(i) for i in two.split())
    return three

In [12]:
df2 = df.drop(["_id","end","series","start","url","airdate"], axis=1)

In [13]:
text = df2.applymap(cleaning)['raw_text']
text_list = [i.split() for i in text]
len(text_list)

678

In [14]:
from time import time
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO,
                   filename='running.log',filemode='w')

In [15]:
df = None
df2 = None
docs = None
del df
del df2
del docs
gc.collect()

14

In [16]:
# gc.get_objects()

In [17]:
# Importing Gensim
import gensim
from gensim import corpora

# Creating the term dictionary of our courpus, where every unique term is assigned an index. dictionary = corpora.Dictionary(doc_clean)
dictionary = corpora.Dictionary(text_list)
dictionary.save('dictionary.dict')
print(dictionary)

Dictionary(46254 unique tokens: ['story', 'july', 'expanding', 'infinity', 'light']...)


In [18]:
# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in text_list]
corpora.MmCorpus.serialize('corpus.mm', doc_term_matrix)

# print(len(doc_term_matrix))
# print(doc_term_matrix[100])

Let's see what 10 topics could be:

In [24]:
start = time()
# Creating the object for LDA model using gensim library
# Lda = gensim.models.ldamodel.LdaModel
Lda = gensim.models.LdaMulticore

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=10, id2word = dictionary, passes=50, workers=14)
print('used: {:.2f}s'.format(time()-start))

Process ForkPoolWorker-75:
Process ForkPoolWorker-77:
Process ForkPoolWorker-67:
Process ForkPoolWorker-68:
Process ForkPoolWorker-65:
Process ForkPoolWorker-74:
Process ForkPoolWorker-76:
Process ForkPoolWorker-78:
Process ForkPoolWorker-66:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/proces

KeyboardInterrupt: 

  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/home/aleksod/anaconda3/lib/python3.6/site-packages/gensim/models/ldamulticore.py", line 274, in worker_e_step
    chunk_no, chunk, worker_lda = input_queue.get()
Traceback (most recent call last):
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/aleksod/anaconda3

  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
Traceback (most recent call last):
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/queues.py", line 93, in get
    with self._rlock:
KeyboardInterrupt
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/synchronize.py", line 96, in __enter__
    return self._semlock.__enter__()
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/pool.py", line 103, in worker
    initializer(*initargs)
  File "/home/aleksod/anaconda3/lib/python3.6/multiprocessing/process.py", 

In [ ]:
with open("Data/ldamodel1.pkl", 'wb') as picklefile:
    pickle.dump(ldamodel, picklefile)

In [ ]:
print(ldamodel.print_topics(num_topics=2, num_words=4))

In [ ]:
for i in ldamodel.print_topics(): 
    for j in i: print(j)

Brilliant! My topics are based on characters that are in the respective series! Not a very useful information, but it LDA works pretty well. Let's see if we can visuzlize it.

In [ ]:
ldamodel.save('topic.model')

In [ ]:
from gensim.models import LdaModel
loading = LdaModel.load('topic.model')

In [ ]:
print(loading.print_topics(num_topics=2, num_words=4))

In [ ]:
def pre_new(doc):
    one = cleaning(doc).split()
    two = dictionary.doc2bow(one)
    return two

In [ ]:
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()

In [ ]:
d = gensim.corpora.Dictionary.load('dictionary.dict')
c = gensim.corpora.MmCorpus('corpus.mm')
lda = gensim.models.LdaModel.load('topic.model')

In [ ]:
data = pyLDAvis.gensim.prepare(lda, c, d)
data

In [ ]:
pyLDAvis.save_html(data,'vis1.html')

This is a pretty good separation. Unfortunately, it is based mostly on very specific TV show / movies terms. Therefore, I will need to remove those if I want better results.